So now that we have our positional data, we want to figure out where to open brick-and-motar locations for AGM. Our approach for identifying good locations to open is to open stores near the most central locations in each county. 

The point of this notebook is demostrating how Neo4j can be used to find the most central points taking into account the distance from other locations and potentially the expected population at each. Furthermore, once we know these locations we will need to identify the closest BART station and plot a course from the station to the central points.

First we will import the data on BART Stations and locations into NOSQL tables.

In [1]:
# Run this to install necessary packages and restart the kernel for osmnx import error to be resolved

# !pip install geopandas
# !pip install folium
# !pip install geopy
# !pip install taxicab
# !pip install osmnx==1.9.1
# !pip install --upgrade osmnx matplotlib numpy

In [2]:
import neo4j
import pandas as pd
from IPython.display import display

import warnings

warnings.simplefilter("ignore", category=FutureWarning)

import geopandas as gpd
import math
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from folium.features import DivIcon
import requests 
import numpy as np
import random
from geopy.geocoders import Nominatim
from IPython.display import Image, IFrame 
from IPython.core.display import HTML 
from pandas import json_normalize
%matplotlib inline
import osmnx as ox
from osmnx import plot_route_folium
import networkx as nx
import matplotlib
import matplotlib as plt
import taxicab as tc
from IPython.display import clear_output
import csv

/opt/conda/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/opt/conda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
locations_df = pd.read_csv("locations_project.csv")

locations_df.head()

,name,county,coordinates,esti. pop.,nearest node one,nearest node 2,nearest node 3,distance 1,distance 2,distance 3
0,Fr Dental Laboratory,San Mateo,"(37.485814, -122.238715)",2651,"(37.478609, -122.291883)","(37.444031, -122.161216)","(37.491431, -122.228803)",5629.155382,7902.135854,1246.209503
1,Pacifica Spindrift Players,San Mateo,"(37.592423, -122.489007)",306,"(37.459552, -122.429672)","(37.462736, -122.429145)","(37.633669, -122.397713)",21589.095445,21269.789077,15705.573372
2,Pulgas Ridge Open Space Preserve,San Mateo,"(37.478609, -122.291883)",1745,"(37.485814, -122.238715)","(37.513253, -122.33258)","(37.491431, -122.228803)",5606.499382,8101.156342,6747.760637
3,GoPro,San Mateo,"(37.534532, -122.331311)",494,"(37.513253, -122.33258)","(37.535644, -122.334809)","(37.545193, -122.306168)",4426.607702,2830.915494,5630.481052
4,Forensic Laboratory of San Mateo Co. Sheriff's...,San Mateo,"(37.513253, -122.33258)",2878,"(37.534532, -122.331311)","(37.535644, -122.334809)","(37.545193, -122.306168)",4426.607702,3428.965861,5643.734418


In [4]:
bart_stations_df = pd.read_csv("BART_Stations_project.csv")

bart_stations_df.head()

,name,county,coordinates,esti. pop.
0,West Dublin/Pleasanton BART Station,Contra Costa,"(37.699709, -121.928243)",428
1,Montgomery St BART Station,San Francisco,"(37.788862, -122.402116)",1812
2,San Antonio BART Station,Alameda,"(37.788309, -122.245649)",1489
3,Fremont BART Station,Alameda,"(37.557451, -121.976611)",1813
4,Santana Row BART Station,Santa Clara,"(37.323154, -121.948811)",965


Dimensions of our data:

In [5]:
print(f"locations_df: {locations_df.shape}")
print(f"bart_stations_df: {bart_stations_df.shape}")

locations_df: (100, 10)
bart_stations_df: (18, 4)


Split the locations df into 2 dataframes, one defining each node and the other describing relations within set of nodes

In [6]:
def get_nodes(county):

    # Split locations into two dataframes

    nodes = locations_df[['name', "county", "coordinates", "esti. pop." ]].loc[locations_df['county'] == county]
    node_relations = locations_df[["coordinates", "nearest node one", "nearest node 2", "nearest node 3", "distance 1", "distance 2", "distance 3"]].loc[locations_df['county'] == county]
    
    return nodes, node_relations

nodes, node_relations = get_nodes('Santa Clara')

In [7]:
nodes

,name,county,coordinates,esti. pop.
60,Victory Martial Arts,Santa Clara,"(37.244966, -121.889866)",432
61,Multitest Electronic Systems,Santa Clara,"(37.377044, -121.95603)",1913
62,SYNQMINE Business Intelligence & Outsource CFO...,Santa Clara,"(37.402767, -121.891175)",2059
63,Lucas Hall,Santa Clara,"(37.351107, -121.939733)",436
64,Benefit Brow Bar At Ulta,Santa Clara,"(37.249065, -121.801793)",894
65,Hot Java To Go,Santa Clara,"(37.214525, -121.727838)",744
66,Intel Santa Clara - SC2,Santa Clara,"(37.375963, -121.975229)",2033
67,Miller Networks,Santa Clara,"(37.116172, -121.640973)",1344
68,Skip's Tire & Auto Center - Cottle Road,Santa Clara,"(37.244915, -121.804446)",1389
69,Club Hitachi Gym,Santa Clara,"(37.373002, -121.947864)",391


In [8]:
def augment_node_relations(nodes, node_relations):

    # We can map each Lat. long. pair key 

    HT = {}

    for i in range(nodes.shape[0]):
        HT[nodes.iloc[i]['coordinates']] = nodes.iloc[i]['name']

    # Next we need to replace the coordinates in the node relations table

    origin_node = []
    nn_one = []
    nn_two = []
    nn_three = []

    for i in range(node_relations.shape[0]):
        origin_node.append(HT[node_relations.iloc[i]['coordinates']])
        nn_one.append(HT[node_relations.iloc[i]['nearest node one']])
        nn_two.append(HT[node_relations.iloc[i]['nearest node 2']])
        nn_three.append(HT[node_relations.iloc[i]['nearest node 3']])

    node_relations['origin_node'] = origin_node
    node_relations['Nearest Node 1 Name'] = nn_one
    node_relations['Nearest Node 2 Name'] = nn_two
    node_relations['Nearest Node 3 Name'] = nn_three

    return node_relations

node_relations = augment_node_relations(nodes, node_relations)

In [9]:
def generate_query(nodes, node_relations):
    
    # We will need to store the original and new names
    
    transformed_names = {}
    
    # We want to create the table for Neo4j

    create_query = "CREATE\n"

    for i in range(nodes.shape[0]):

        loc_name = nodes.iloc[i]['name'].replace(':', '').replace('&', '').replace(',', '').replace('.', '_').replace('-', '').replace("'", "").replace("#","").replace("/","").replace(' ', '')

        try:

            if type(int(loc_name[0])) == int:
                create_query += f"\t(l_{loc_name.lower()}:POINT" + " {" + f"name:'l_{loc_name}', "  + f" population: {nodes.iloc[i]['esti. pop.']}" + "}),\n" # + f"latitude: {nodes.iloc[i]['coordinates'].strip('()').split(', ')[0]}," + f" longitude: {nodes.iloc[i]['coordinates'].strip('()').split(', ')[1]},"
                transformed_names[f"l_{loc_name}"] = nodes.iloc[i]['name']
                
        except:
            create_query += f"\t({loc_name.lower()}:POINT" + " {" + f"name:'{loc_name}', "  + f" population: {nodes.iloc[i]['esti. pop.']}" + "}),\n" # + f"latitude: {nodes.iloc[i]['coordinates'].strip('()').split(', ')[0]}," + f" longitude: {nodes.iloc[i]['coordinates'].strip('()').split(', ')[1]},"
            transformed_names[loc_name] = nodes.iloc[i]['name']
            
    for i in range(node_relations.shape[0]):

        origin_name = node_relations.iloc[i]['origin_node'].replace(':', '').replace('&', '').replace(',', '').replace('.', '_').replace('-', '').replace("'", "").replace("#","").replace("/","").replace(' ', '')

        nn_1_name = node_relations.iloc[i]['Nearest Node 1 Name'].replace(':', '').replace('&', '').replace(',', '').replace('.', '_').replace('-', '').replace("'", "").replace("#","").replace("/","").replace(' ', '')
        nn_2_name = node_relations.iloc[i]['Nearest Node 2 Name'].replace(':', '').replace('&', '').replace(',', '').replace('.', '_').replace('-', '').replace("'", "").replace("#","").replace("/","").replace(' ', '')
        nn_3_name = node_relations.iloc[i]['Nearest Node 3 Name'].replace(':', '').replace('&', '').replace(',', '').replace('.', '_').replace('-', '').replace("'", "").replace("#","").replace("/","").replace(' ', '')

        try:
            if type(int(origin_name[0])) == int:
                origin_name = "l_" + origin_name

        except:
            pass

        try:
            if type(int(nn_1_name[0])) == int:
                nn_1_name = "l_" + nn_1_name

        except:
            pass

        try:
            if type(int(nn_2_name[0])) == int:
                nn_2_name = "l_" + nn_2_name

        except:
            pass

        try:
            if type(int(nn_3_name[0])) == int:
                nn_3_name = "l_" + nn_3_name

        except:
            pass

        create_query += f"\t({origin_name.lower()})" + "-[:distance {" + f"distance: {node_relations.iloc[i]['distance 1']}" + "}]-> " + f"({nn_1_name.lower()}),\n"
        create_query += f"\t({origin_name.lower()})" + "-[:distance {" + f"distance: {node_relations.iloc[i]['distance 2']}" + "}]-> " + f"({nn_2_name.lower()}),\n"
        create_query += f"\t({origin_name.lower()})" + "-[:distance {" + f"distance: {node_relations.iloc[i]['distance 3']}" + "}]-> " + f"({nn_3_name.lower()})"
    #     (seattle)-[:TRACK {track_miles: 798}]->(berkeley),
        if i < node_relations.shape[0] - 1:
            create_query += ",\n"

    return create_query, transformed_names

create_query, transformed_names = generate_query(nodes, node_relations)


Web server interface at https://xxxx:7473
Username: neo4j
Password: ucb_mids_w205

The next step is to initialize Neo4j for our notebook (We are using code from Lab 9)

In [10]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [11]:
session = driver.session(database="neo4j")

In [12]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [13]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [14]:
def my_neo4j_nodes_relationships():
    "print all the nodes and relationships"
   
    print("-------------------------")
    print("  Nodes:")
    print("-------------------------")
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    display(df)
    
    print("-------------------------")
    print("  Relationships:")
    print("-------------------------")
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    display(df)
    
    density = (2 * number_relationships) / (number_nodes * (number_nodes - 1))
    
    print("-------------------------")
    print("  Density:", f'{density:.1f}')
    print("-------------------------")

In [15]:
def my_create_connected_graph(create_query):
    "create the connected graph"
    
    my_neo4j_wipe_out_database()

    session.run(create_query)

In [16]:
my_create_connected_graph(create_query)

In [17]:
my_neo4j_nodes_relationships()

-------------------------
  Nodes:
-------------------------


,node_name,labels
0,BenefitBrowBarAtUlta,[POINT]
1,BloomFresh,[POINT]
2,CaringNeighborsEnterprisesInc_,[POINT]
3,ChildrensDiscoveryMuseumofSanJose,[POINT]
4,ClubHitachiGym,[POINT]
5,CrystallineSoundHealing,[POINT]
6,HotJavaToGo,[POINT]
7,IntelSantaClaraSC2,[POINT]
8,KineticWellnessCenterDrGeorgeChiropracticCorpo...,[POINT]
9,LucasHall,[POINT]


-------------------------
  Relationships:
-------------------------


,node_name_1,node_1_labels,relationship_type,node_name_2,node_2_labels
0,BenefitBrowBarAtUlta,[POINT],distance,OCZTechnologyGroup,[POINT]
1,BenefitBrowBarAtUlta,[POINT],distance,PrimetimeMartialArts,[POINT]
2,BenefitBrowBarAtUlta,[POINT],distance,SkipsTireAutoCenterCottleRoad,[POINT]
3,BloomFresh,[POINT],distance,ChildrensDiscoveryMuseumofSanJose,[POINT]
4,BloomFresh,[POINT],distance,SALClassrooms,[POINT]
5,BloomFresh,[POINT],distance,SYNQMINEBusinessIntelligenceOutsourceCFOServices,[POINT]
6,CaringNeighborsEnterprisesInc_,[POINT],distance,ClubHitachiGym,[POINT]
7,CaringNeighborsEnterprisesInc_,[POINT],distance,MultitestElectronicSystems,[POINT]
8,CaringNeighborsEnterprisesInc_,[POINT],distance,SYNQMINEBusinessIntelligenceOutsourceCFOServices,[POINT]
9,ChildrensDiscoveryMuseumofSanJose,[POINT],distance,BloomFresh,[POINT]


-------------------------
  Density: 0.3
-------------------------


In [18]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'POINT', {distance: {properties: 'distance'}})"
session.run(query)

In [19]:
query = """

CALL gds.betweenness.stream('ds_graph', {relationshipWeightProperty: 'distance'})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as betweenness
ORDER BY betweenness DESC

"""

res = my_neo4j_run_query_pandas(query)

res

,name,betweenness
0,PrimetimeMartialArts,16.0
1,LucasHall,14.0
2,OCZTechnologyGroup,13.0
3,ClubHitachiGym,7.0
4,VictoryMartialArts,6.0
5,IntelSantaClaraSC2,5.0
6,ChildrensDiscoveryMuseumofSanJose,5.0
7,BloomFresh,4.0
8,KineticWellnessCenterDrGeorgeChiropracticCorpo...,3.0
9,SYNQMINEBusinessIntelligenceOutsourceCFOServices,3.0


In [20]:
res.iloc[0]

name           PrimetimeMartialArts
betweenness                    16.0
Name: 0, dtype: object

In [21]:
# my_neo4j_wipe_out_database()

We need to know the point in each network with the highest centrality

In [22]:
county_central = {}

for county in set(locations_df['county']):
    
    nodes, node_relations = get_nodes(county)
    
    node_relations = augment_node_relations(nodes, node_relations)
    
    create_query, transformed_names = generate_query(nodes, node_relations)
    
    my_create_connected_graph(create_query)
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'POINT', {distance: {properties: 'distance'}})"
    session.run(query)
    
    query = """

    CALL gds.betweenness.stream('ds_graph', {relationshipWeightProperty: 'distance'})
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS name, score as betweenness
    ORDER BY betweenness DESC

    """    
    res = my_neo4j_run_query_pandas(query)
    
    # Augment res with matching coordinates
    
    coord = []
    
    for i in range(res.shape[0]):
        
        temp = locations_df[['coordinates']].loc[locations_df['name'] == transformed_names[res.iloc[i]['name']] ]
        
        coord.append(temp.iat[0,0])
    
    res['coordinates'] = coord
    
    file_name = f'{county} Betweeness Results'

    res.to_csv(file_name, sep='\t')
    
#     print(locations_df.loc[locations_df['name'] == transformed_names[res.iloc[0]['name']] ]['coordinates'].iloc[0])

    county_central[county] = (res.iloc[0]['name'], locations_df.loc[locations_df['name'] == transformed_names[res.iloc[0]['name']] ]['coordinates'].iloc[0] )
    
county_central

{'Santa Clara': ('PrimetimeMartialArts', '(37.236254, -121.805053)'),
 'Contra Costa': ('NewhallCommunityPark', '(37.953737, -121.977269)'),
 'San Mateo': ('ForensicLaboratoryofSanMateoCo_SheriffsOffice',
  '(37.513253, -122.33258)'),
 'Alameda': ('PhantomMotors', '(37.678241, -122.09081)'),
 'San Francisco': ('OldWestWesternWear', '(37.754265, -122.418352)')}

In [23]:
# Initialze the various maps we will need ahead of time to save on computing

maps = {}

for county in set(locations_df['county']):
    maps[county] = ox.graph_from_place(f"{county}, California, USA", network_type="drive")
    
print("Done!")

Done!


In [24]:
# Declare arrays to keep track of each other node's closest BART Station

nearest_station_names = []
nearest_station_coordinates = []
nearest_distances = []

for i, county in enumerate(set(locations_df['county'])):
    
    temp_nearest_station_name = ""
    temp_nearest_station_coordinate = (0,0)
    temp_nearest_distance = 100000000
    
    origin_coordinates = county_central[county][1]
    location_name = county_central[county][0]
    
    G = maps[county]
    
    for j in range(bart_stations_df.shape[0]):
        
        destination_coordinates = bart_stations_df.iloc[j]['coordinates']
        
        orig = origin_coordinates.strip("()").split(", ")
        dest = destination_coordinates.strip("()").split(", ")
        
        orig = (float(orig[0]), float(orig[1]))
        dest = (float(dest[0]), float(dest[1]))
    
        try:
            
            route = tc.distance.shortest_path(G, orig, dest)
            
            distance = route[0]

            if distance != 0 and distance < temp_nearest_distance:

                temp_nearest_distance = distance
                temp_nearest_station_name = bart_stations_df.iloc[j]['name']
                temp_nearest_station_coordinate = dest

        except:
            pass
        
        clear_output(wait=True)
        print(f"{i + 1} out of {len(set(locations_df['county']))} counties | node {j + 1} | {j + 1}/{bart_stations_df.shape[0]} of  distances calculated")
        
    nearest_station_names.append([county, temp_nearest_station_name])
    nearest_station_coordinates.append(temp_nearest_station_coordinate)
    nearest_distances.append(temp_nearest_distance)

5 out of 5 counties | node 18 | 18/18 of  distances calculated


In [25]:
nearest_station_names

[['Santa Clara', 'Santana Row BART Station'],
 ['Contra Costa', 'Oak Grove BART Station'],
 ['San Mateo', 'Lafayette BART Station'],
 ['Alameda', 'Hayward BART Station'],
 ['San Francisco', 'Glen Park BART Station']]

In [26]:
nearest_distances

[19841.352105753722,
 6502.00362830697,
 17035.596304528794,
 1065.776602003451,
 3025.406207696406]

In [27]:
nearest_station_coordinates

[(37.323154, -121.948811),
 (37.943057, -122.029329),
 (37.893161, -122.124925),
 (37.669748, -122.087035),
 (37.733119, -122.433797)]

Now we want to visualize the points in each county. There are some key features which must be present:

- All nodes in the county
- The central node must be a different color
- The BART station must be a different color
- The path from the BART Station to the central node must be clearly highlighted

In [28]:
# Lets start by initializing all maps

counties = ['San Francisco', 'Santa Clara', 'Contra Costa', 'Alameda', 'San Mateo']
coordinates = [[37.759344, -122.446452], [37.258336, -121.734320], [37.955321, -121.879418], 
               [37.674112, -121.971002], [37.466796, -122.391435]]

# Declare a Hash Table to keep track of the maps

folium_maps = {}

for i, county in enumerate(counties):
    folium_maps[county] = folium.Map(location=coordinates[i], tiles='openstreetmap', prefer_canvas=True)

# --------------------------------------------------------------------------------------------
    
# Auxiliary function for plotting points 

def plot_point(node, map_, station=False, station_name=""):
    
    
    if station == False:
        
        point = node['coordinates'].strip("()").split(", ")

        point = (float(point[0]), float(point[1]))
        
        folium.CircleMarker(location=[point[0], point[1]],
                            radius=2,
                            weight=5,
                           color='blue',
                           label = node['name']).add_to(map_)
        folium.map.Marker(location=[point[0], point[1]],
                      icon=DivIcon(
                          icon_size=(20,20),
                          icon_anchor=(5,14),
                          html=f'<div style="font-size: 7pt">%s</div>' % node['name'],
                      )
                     ).add_to(map_)
    else:
        
        point = str(node).strip("()").split(", ")

        point = (float(point[0]), float(point[1]))
        
        folium.CircleMarker(location=[point[0], point[1]],
                            radius=2,
                            weight=5,
                           color='red',
                           name=station_name).add_to(map_)
        folium.map.Marker(location=[point[0], point[1]],
                      icon=DivIcon(
                          icon_size=(20,20),
                          icon_anchor=(5,14),
                          html=f'<div style="font-size: 7pt">%s</div>' % station_name,
                      )
                     ).add_to(map_)

# Auxiliary function for plotting shortest paths

def plot_path(orig, dest, county, map_):
    
    orig = orig.strip("()").split(", ")
    orig = (float(orig[0]), float(orig[1]))
    
#     print(orig[0], dest)
    
    G = maps[county]
#     G = map_
    
    origin_node = ox.distance.nearest_nodes(G, X=orig[1], Y=orig[0])
    destination_node = ox.distance.nearest_nodes(G, X=dest[1], Y=dest[0])
    
    route = nx.shortest_path(G, origin_node, destination_node)
    
    map_ = ox.plot_route_folium(G, route, route_map=map_, weight=10)

# -------------------------------------------------------------------------------------------------------------
    
# Plot all features for all maps

# Plot Nodes 
for county in set(locations_df['county']):

    nodes, node_relations = get_nodes(county)
    
    for i in range(nodes.shape[0]):
              
        plot_point(nodes.iloc[i], folium_maps[county])

# Plot Routes From BART to Central Node
for county in set(locations_df['county']):
    
    index = 0
    
    for i, entry in enumerate(nearest_station_names):
        
        if entry[0] == county:
            index = i 
            break
            
    dest = nearest_station_coordinates[index]
    orig = county_central[county][1]
    
    plot_path(orig, dest, county, folium_maps[county])

# Plot BART Nodes
for i, entry in enumerate(nearest_station_names):
    
    county = entry[0]
    
    Station_Name = entry[1]
    
    orig = nearest_station_coordinates[i]
    
    plot_point(orig, folium_maps[county], station=True, station_name=Station_Name)

    folium_maps[county].fit_bounds(folium_maps[county].get_bounds())
 
    filepath = f'{county}_route_graph.html'
    folium_maps[county].save(filepath)

In [29]:
for i, county in enumerate(counties):
    
    print(county)
    display(folium_maps[county])

San Francisco


Santa Clara


Contra Costa


Alameda


San Mateo
